In [1]:
import pdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sklearn
import statsmodels.formula.api as sm
import random
import itertools

In [3]:
mu,sigma=0.0,1.0
def create_random(num):
    pd_rondom=np.random.normal(mu, sigma, num)
    pd_rondom=pd.DataFrame(pd_rondom)
    return pd_rondom
# return x depending on element num and data num
def make_x(element_num,data_num):
    x_matrix=create_random(element_num)
    for i in range(data_num-1):
        x_small=create_random(element_num) #train_w = pd.DataFrame(fixed_random)[0:100]
        x_matrix = pd.concat([x_matrix,x_small],axis=1) #pd.concat([create_random(100),create_random(100)],axis=1)
    X = x_matrix.T
    X.index = np.linspace(0,len(X.index)-1,len(X.index))
    X.columns= np.linspace(0,len(X.columns)-1,len(X.columns))
    return X

In [4]:
#"""""""""""""""""""""""""""""""""""""""
#1.1
#""""""""""""""""""""""""""""""""""""""""

In [5]:
#######
# DATA
# K
#

element_num,data_num = 3,10
k=5
X = make_x(element_num,data_num)


In [6]:
# This function is used for only the first time of algorithm
def initial_indicator(X,k):
    r_indicator = pd.DataFrame(np.zeros([len(X),k]))
    clusters = np.linspace(0,k-1,k,dtype=int)
    data_num = np.linspace(0,len(X)-1,len(X),dtype=int)
    
    for cl in clusters:
        random_datum = np.random.choice(data_num,size=1)
        r_indicator.iloc[random_datum,cl]=1
        del_indice = np.where(data_num == random_datum)
        data_num = np.delete(data_num,del_indice)

    for da in data_num:
        random_cluster = np.random.choice(clusters,size=1)
        r_indicator.iloc[da,random_cluster]=1
    return r_indicator


In [7]:
# Return the new clustering 
def assign_data(X,r_indicator):   
      # for every data x_i
    for X_row in range(len(X)):
        distance_list = []
        for cluster in r_indicator.columns:
            not_0_index = r_indicator[cluster].ix[r_indicator[cluster]!=0].index
            centre_cluster = X.ix[not_0_index].mean(axis=0)  ## recomputing cluster center        
            diff = (X.ix[X_row] - centre_cluster)
            distance_list.append(diff.dot(diff.T))

        new_j = np.argmin(distance_list) # new cluster index for data X_row
        r_indicator.iloc[X_row,:]=0
        r_indicator.iloc[X_row,new_j]=1
        
    return r_indicator
    

In [8]:
############################
#
#   implementing  
#        1. assign data 
#        2.check any change
#        3. repeat
############################

r_indicator = initial_indicator(X,k)
print "first",r_indicator

while True:  # if there is any change, continue !(new_r_indicator == r_indicator):
    new_r_indicator = assign_data(X,r_indicator)
    if (new_r_indicator.values == r_indicator.values).all():break
    else: continue

print "end",new_r_indicator

first    0  1  2  3  4
0  0  0  0  1  0
1  0  1  0  0  0
2  0  1  0  0  0
3  0  0  0  0  1
4  0  1  0  0  0
5  1  0  0  0  0
6  0  0  0  0  1
7  0  0  0  0  1
8  0  0  1  0  0
9  1  0  0  0  0
end    0  1  2  3  4
0  0  0  0  1  0
1  1  0  0  0  0
2  0  0  0  0  1
3  0  1  0  0  0
4  0  1  0  0  0
5  1  0  0  0  0
6  0  0  0  0  1
7  0  0  0  0  1
8  0  0  1  0  0
9  1  0  0  0  0


In [9]:
#"""""""""""""""""""""""""""""""""""""""
#1.1
#    2.
#""""""""""""""""""""""""""""""""""""""""

In [10]:
# This is the provided code but it does not think cholesky.
# the two variable normal distribution is not expressed from just a sum product.
def genData2():
    A1 = np.matrix([[0.29,0.4],[0.4,0.4]])
    u1 = np.matrix([4,0])
    A2 = np.matrix([[0.29,0.6],[0.6,0.9]])
    u2 = np.matrix([5,7])
    A3 = np.matrix([[0.64,0],[0,0.64]])
    u3 = np.matrix([7,4])
    data = np.random.randn(150,2)
    data =np.matrix(data)
    for i in range(50):
        data[i,:] = (u1.T + A1.dot(data[i,:].T)).T
    for i in range(50):
        data[50+i,:] = ( u2.T + A2.dot(data[50+i,:].T) ).T
    for i in range(50):
        data[100+i,:] = ( u3.T + A3.dot(data[100+i,:].T) ).T
    return data


In [11]:
# This is the code to generate random data. 
# This think cholesky decomposition.
def genData2_2():
    mean1 = [4.0,0.0]
    cov1 = [[0.29,0.4],[0.4,0.4]]
    mean2 = [5.0,7.0]
    cov2 = [[0.29,0.6],[0.6,0.9]]
    mean3 = [7.0,4.0]
    cov3 = [[0.64,0.0],[0.0,0.64]]  

    data1 = np.random.multivariate_normal(mean1, cov1, 50  )
    data2 = np.random.multivariate_normal(mean2, cov2, 50   )
    data3 = np.random.multivariate_normal(mean3, cov3, 50   )
    
    data = np.concatenate((data1, data2,data3), axis=0)
    data =np.matrix(data)
    
    return data

In [12]:
data =genData2() 
#data = genData2_2()
plt.scatter(data[0:49,0],data[0:49,1],color='blue',marker=u'x',s=15)
plt.scatter(data[0:49,0].mean(),data[0:49,1].mean(),color='blue',marker=u'o',s=50)

plt.scatter(data[50:99,0],data[50:99,1],color="red",marker=u'x',s=15)
plt.scatter(data[50:99,0].mean(),data[50:99,1].mean(),color="red",marker=u'o',s=50)

plt.scatter(data[100:149,0],data[100:149,1],color="green",marker=u'x',s=15)
plt.scatter(data[100:149,0].mean(),data[100:149,1].mean(),color="green",marker=u'o',s=50)

plt.legend(("S1","S1 center","S2","S2 center","S3","S3 center"))
plt.xlabel("x")
plt.ylabel("y")
#plt.title("data from gendata")
plt.show()

In [13]:
#####################################
#
#  last of 2. 
#

In [14]:
import itertools
k=3
seq = np.linspace(0,k-1,k)
C_list = list(itertools.permutations(seq))

def kmean_1time_return_occe(X,k,C_list):
    r_indicator = initial_indicator(X,k)
    while True: #kotonatte iru aida !(new_r_indicator == r_indicator):
        new_r_indicator = assign_data(X,r_indicator)
        if (new_r_indicator.values == r_indicator.values).all():break
        else: continue
    
    error_list_sum_of_xi = np.zeros(len(C_list))
    error_list_norm_of_x = np.zeros(len(C_list))
    for C_num in range(len(C_list)):
        C = C_list[C_num]
        correct_answer = pd.DataFrame(np.zeros(new_r_indicator.shape))
        correct_answer.iloc[0:49,int(C[0])]=1
        correct_answer.iloc[50:99,int(C[1])]=1
        correct_answer.iloc[100:len(new_r_indicator),int(C[2])]=1

        tru_fal_matrix = (correct_answer==new_r_indicator)
        ones = np.bool(len(tru_fal_matrix))
        a = np.array(tru_fal_matrix.T.all())
        tru_fal_index = ones - a

        error_data = X.ix[tru_fal_index]
        l = len(X)*1.0
        err = 0
        for i in (error_data.index):
            err += np.linalg.norm(error_data.ix[i])
        error_list_sum_of_xi[C_num] = err/l
        error_list_norm_of_x[C_num] = np.linalg.norm(error_data)/l

    return error_list_norm_of_x[error_list_norm_of_x.argmin()]

data=genData2()  # set the data
occe_history = [] 
repeat_num = 100
data=pd.DataFrame(data)
for i in range(repeat_num):
    occe = kmean_1time_return_occe(data,k,C_list)
    occe_history.append(occe)
        

In [15]:
print "mean",np.array(occe_history).mean()
print "s.d.", ( np.array(occe_history).var()  )**0.5

mean 0.0763441823573
s.d. 0.0292949245386


In [16]:
##############################
#
#   IRIS
#

In [17]:
columns=[u'sepal_len', u'sepal_wid', u'petal_len', u'petal_wid', u'type']

In [18]:
index =np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149])

In [19]:
# THis data is the iris data
iris=np.array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
       [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
       [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
       [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'],
       [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
       [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
       [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
       [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
       [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'],
       [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
       [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
       [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
       [4.6, 3.6, 1.0, 0.2, 'Iris-setosa'],
       [5.1, 3.3, 1.7, 0.5, 'Iris-setosa'],
       [4.8, 3.4, 1.9, 0.2, 'Iris-setosa'],
       [5.0, 3.0, 1.6, 0.2, 'Iris-setosa'],
       [5.0, 3.4, 1.6, 0.4, 'Iris-setosa'],
       [5.2, 3.5, 1.5, 0.2, 'Iris-setosa'],
       [5.2, 3.4, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.6, 0.2, 'Iris-setosa'],
       [4.8, 3.1, 1.6, 0.2, 'Iris-setosa'],
       [5.4, 3.4, 1.5, 0.4, 'Iris-setosa'],
       [5.2, 4.1, 1.5, 0.1, 'Iris-setosa'],
       [5.5, 4.2, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [5.0, 3.2, 1.2, 0.2, 'Iris-setosa'],
       [5.5, 3.5, 1.3, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [4.4, 3.0, 1.3, 0.2, 'Iris-setosa'],
       [5.1, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.5, 1.3, 0.3, 'Iris-setosa'],
       [4.5, 2.3, 1.3, 0.3, 'Iris-setosa'],
       [4.4, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [5.0, 3.5, 1.6, 0.6, 'Iris-setosa'],
       [5.1, 3.8, 1.9, 0.4, 'Iris-setosa'],
       [4.8, 3.0, 1.4, 0.3, 'Iris-setosa'],
       [5.1, 3.8, 1.6, 0.2, 'Iris-setosa'],
       [4.6, 3.2, 1.4, 0.2, 'Iris-setosa'],
       [5.3, 3.7, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.3, 1.4, 0.2, 'Iris-setosa'],
       [7.0, 3.2, 4.7, 1.4, 'Iris-versicolor'],
       [6.4, 3.2, 4.5, 1.5, 'Iris-versicolor'],
       [6.9, 3.1, 4.9, 1.5, 'Iris-versicolor'],
       [5.5, 2.3, 4.0, 1.3, 'Iris-versicolor'],
       [6.5, 2.8, 4.6, 1.5, 'Iris-versicolor'],
       [5.7, 2.8, 4.5, 1.3, 'Iris-versicolor'],
       [6.3, 3.3, 4.7, 1.6, 'Iris-versicolor'],
       [4.9, 2.4, 3.3, 1.0, 'Iris-versicolor'],
       [6.6, 2.9, 4.6, 1.3, 'Iris-versicolor'],
       [5.2, 2.7, 3.9, 1.4, 'Iris-versicolor'],
       [5.0, 2.0, 3.5, 1.0, 'Iris-versicolor'],
       [5.9, 3.0, 4.2, 1.5, 'Iris-versicolor'],
       [6.0, 2.2, 4.0, 1.0, 'Iris-versicolor'],
       [6.1, 2.9, 4.7, 1.4, 'Iris-versicolor'],
       [5.6, 2.9, 3.6, 1.3, 'Iris-versicolor'],
       [6.7, 3.1, 4.4, 1.4, 'Iris-versicolor'],
       [5.6, 3.0, 4.5, 1.5, 'Iris-versicolor'],
       [5.8, 2.7, 4.1, 1.0, 'Iris-versicolor'],
       [6.2, 2.2, 4.5, 1.5, 'Iris-versicolor'],
       [5.6, 2.5, 3.9, 1.1, 'Iris-versicolor'],
       [5.9, 3.2, 4.8, 1.8, 'Iris-versicolor'],
       [6.1, 2.8, 4.0, 1.3, 'Iris-versicolor'],
       [6.3, 2.5, 4.9, 1.5, 'Iris-versicolor'],
       [6.1, 2.8, 4.7, 1.2, 'Iris-versicolor'],
       [6.4, 2.9, 4.3, 1.3, 'Iris-versicolor'],
       [6.6, 3.0, 4.4, 1.4, 'Iris-versicolor'],
       [6.8, 2.8, 4.8, 1.4, 'Iris-versicolor'],
       [6.7, 3.0, 5.0, 1.7, 'Iris-versicolor'],
       [6.0, 2.9, 4.5, 1.5, 'Iris-versicolor'],
       [5.7, 2.6, 3.5, 1.0, 'Iris-versicolor'],
       [5.5, 2.4, 3.8, 1.1, 'Iris-versicolor'],
       [5.5, 2.4, 3.7, 1.0, 'Iris-versicolor'],
       [5.8, 2.7, 3.9, 1.2, 'Iris-versicolor'],
       [6.0, 2.7, 5.1, 1.6, 'Iris-versicolor'],
       [5.4, 3.0, 4.5, 1.5, 'Iris-versicolor'],
       [6.0, 3.4, 4.5, 1.6, 'Iris-versicolor'],
       [6.7, 3.1, 4.7, 1.5, 'Iris-versicolor'],
       [6.3, 2.3, 4.4, 1.3, 'Iris-versicolor'],
       [5.6, 3.0, 4.1, 1.3, 'Iris-versicolor'],
       [5.5, 2.5, 4.0, 1.3, 'Iris-versicolor'],
       [5.5, 2.6, 4.4, 1.2, 'Iris-versicolor'],
       [6.1, 3.0, 4.6, 1.4, 'Iris-versicolor'],
       [5.8, 2.6, 4.0, 1.2, 'Iris-versicolor'],
       [5.0, 2.3, 3.3, 1.0, 'Iris-versicolor'],
       [5.6, 2.7, 4.2, 1.3, 'Iris-versicolor'],
       [5.7, 3.0, 4.2, 1.2, 'Iris-versicolor'],
       [5.7, 2.9, 4.2, 1.3, 'Iris-versicolor'],
       [6.2, 2.9, 4.3, 1.3, 'Iris-versicolor'],
       [5.1, 2.5, 3.0, 1.1, 'Iris-versicolor'],
       [5.7, 2.8, 4.1, 1.3, 'Iris-versicolor'],
       [6.3, 3.3, 6.0, 2.5, 'Iris-virginica'],
       [5.8, 2.7, 5.1, 1.9, 'Iris-virginica'],
       [7.1, 3.0, 5.9, 2.1, 'Iris-virginica'],
       [6.3, 2.9, 5.6, 1.8, 'Iris-virginica'],
       [6.5, 3.0, 5.8, 2.2, 'Iris-virginica'],
       [7.6, 3.0, 6.6, 2.1, 'Iris-virginica'],
       [4.9, 2.5, 4.5, 1.7, 'Iris-virginica'],
       [7.3, 2.9, 6.3, 1.8, 'Iris-virginica'],
       [6.7, 2.5, 5.8, 1.8, 'Iris-virginica'],
       [7.2, 3.6, 6.1, 2.5, 'Iris-virginica'],
       [6.5, 3.2, 5.1, 2.0, 'Iris-virginica'],
       [6.4, 2.7, 5.3, 1.9, 'Iris-virginica'],
       [6.8, 3.0, 5.5, 2.1, 'Iris-virginica'],
       [5.7, 2.5, 5.0, 2.0, 'Iris-virginica'],
       [5.8, 2.8, 5.1, 2.4, 'Iris-virginica'],
       [6.4, 3.2, 5.3, 2.3, 'Iris-virginica'],
       [6.5, 3.0, 5.5, 1.8, 'Iris-virginica'],
       [7.7, 3.8, 6.7, 2.2, 'Iris-virginica'],
       [7.7, 2.6, 6.9, 2.3, 'Iris-virginica'],
       [6.0, 2.2, 5.0, 1.5, 'Iris-virginica'],
       [6.9, 3.2, 5.7, 2.3, 'Iris-virginica'],
       [5.6, 2.8, 4.9, 2.0, 'Iris-virginica'],
       [7.7, 2.8, 6.7, 2.0, 'Iris-virginica'],
       [6.3, 2.7, 4.9, 1.8, 'Iris-virginica'],
       [6.7, 3.3, 5.7, 2.1, 'Iris-virginica'],
       [7.2, 3.2, 6.0, 1.8, 'Iris-virginica'],
       [6.2, 2.8, 4.8, 1.8, 'Iris-virginica'],
       [6.1, 3.0, 4.9, 1.8, 'Iris-virginica'],
       [6.4, 2.8, 5.6, 2.1, 'Iris-virginica'],
       [7.2, 3.0, 5.8, 1.6, 'Iris-virginica'],
       [7.4, 2.8, 6.1, 1.9, 'Iris-virginica'],
       [7.9, 3.8, 6.4, 2.0, 'Iris-virginica'],
       [6.4, 2.8, 5.6, 2.2, 'Iris-virginica'],
       [6.3, 2.8, 5.1, 1.5, 'Iris-virginica'],
       [6.1, 2.6, 5.6, 1.4, 'Iris-virginica'],
       [7.7, 3.0, 6.1, 2.3, 'Iris-virginica'],
       [6.3, 3.4, 5.6, 2.4, 'Iris-virginica'],
       [6.4, 3.1, 5.5, 1.8, 'Iris-virginica'],
       [6.0, 3.0, 4.8, 1.8, 'Iris-virginica'],
       [6.9, 3.1, 5.4, 2.1, 'Iris-virginica'],
       [6.7, 3.1, 5.6, 2.4, 'Iris-virginica'],
       [6.9, 3.1, 5.1, 2.3, 'Iris-virginica'],
       [5.8, 2.7, 5.1, 1.9, 'Iris-virginica'],
       [6.8, 3.2, 5.9, 2.3, 'Iris-virginica'],
       [6.7, 3.3, 5.7, 2.5, 'Iris-virginica'],
       [6.7, 3.0, 5.2, 2.3, 'Iris-virginica'],
       [6.3, 2.5, 5.0, 1.9, 'Iris-virginica'],
       [6.5, 3.0, 5.2, 2.0, 'Iris-virginica'],
       [6.2, 3.4, 5.4, 2.3, 'Iris-virginica'],
       [5.9, 3.0, 5.1, 1.8, 'Iris-virginica']], dtype=object)

In [20]:
iris = pd.DataFrame(iris)
iris.columns=columns
iris.index=index

In [21]:
X = iris.drop(u'type',axis=1)

In [22]:
def kmean_1time_return_occe(X,k,C_list):
    r_indicator = initial_indicator(X,k)
    while True: #kotonatte iru aida !(new_r_indicator == r_indicator):
        new_r_indicator = assign_data(X,r_indicator)
        if (new_r_indicator.values == r_indicator.values).all():break
        else: continue
    
    error_list_sum_of_xi = np.zeros(len(C_list))
    error_list_norm_of_x = np.zeros(len(C_list))
    for C_num in range(len(C_list)):
        C = C_list[C_num]
        correct_answer = pd.DataFrame(np.zeros(new_r_indicator.shape))
        correct_answer.iloc[0:49,int(C[0])]=1
        correct_answer.iloc[50:99,int(C[1])]=1
        correct_answer.iloc[100:len(new_r_indicator),int(C[2])]=1

        tru_fal_matrix = (correct_answer==new_r_indicator)
        ones = np.bool(len(tru_fal_matrix))
        a = np.array(tru_fal_matrix.T.all())
        tru_fal_index = ones - a

        error_data = X.ix[tru_fal_index]
        l = len(X)*1.0
        err = 0
        for i in (error_data.index):
            err += np.linalg.norm(error_data.ix[i])
        error_list_sum_of_xi[C_num] = err/l
        error_list_norm_of_x[C_num] = np.linalg.norm(error_data)/l
    #print "occe_sum_of_xi",error_list_sum_of_xi[error_list_sum_of_xi.argmin()],"correct_cluster",C_list[error_list_sum_of_xi.argmin()]
    #print "occe_norm_of_x",error_list_norm_of_x[error_list_norm_of_x.argmin()],"correct_cluster",C_list[error_list_norm_of_x.argmin()]
    
    #return error_list_sum_of_xi[error_list_sum_of_xi.argmin()]
    return error_list_norm_of_x[error_list_norm_of_x.argmin()]
    
occe_history = []
repeat_num = 100
for i in range(repeat_num):
    occe = kmean_1time_return_occe(X,k,C_list)
    occe_history.append(occe)
        

In [23]:
print "mean",np.array(occe_history).mean()
print "s.d.",( np.array(occe_history).var() )**0.5

mean 0.281234798449
s.d. 0.020131714256
